In [33]:
import threading
import time
import os 
import pandas as pd
import datetime

In [34]:
aggregate_report = pd.read_csv(".//aggr_report.csv")
aggregate = pd.read_csv(".//aggregate.csv")

In [35]:
aggregate["Error %"] = aggregate["Error %"].str.replace("%","")
aggregate["Error %"] = aggregate["Error %"].str.replace(",",".")
aggregate["Error %"] = pd.to_numeric(aggregate["Error %"], downcast="float")
aggregate["Error %"] = aggregate["# Samples"] * aggregate["Error %"] /100
aggr_1 = aggregate[['Label','Throughput','Error %']].copy()
aggr_1.rename(columns = {'Label':'label','Error %' : 'errors'}, inplace = True)

In [36]:
aggregate_report = aggregate_report.groupby(['label'],axis=0,as_index=False).mean()
aggr_2 = aggregate_report[['label','elapsed','Latency']].copy()

In [37]:
aggr_df = pd.merge(aggr_1, aggr_2, on='label')
aggr_df.head()

,label,Throughput,errors,elapsed,Latency
0,01:23:18,"5,50122",0.0,2380.592593,2203.037037
1,01:23:19,"1,97433",0.0,2017.125000,1839.250000
2,01:23:21,"11,23596",0.0,89.000000,7.000000
3,01:23:24,"21,38493",0.0,76.380952,4.238095
4,01:23:25,"14,49275",0.0,121.250000,8.500000


In [ ]:

aggr_df["label"] = aggr_df["label"] + 

In [38]:
if not os.path.isfile('client.csv'):
   aggr_df.to_csv('client.csv',index=False)
else: 
   aggr_df.to_csv('client.csv', index = False,mode='a', header=False)